In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
import numpy as np
import tensorflow as tf
import random

DeepCTR version 0.9.0 detected. Your version is 0.8.7.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


In [2]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

### start training(use 4/1-4/30 model train5/1 data)¶

In [24]:
df_uid_5 = load_variable('/home/jovyan/dataset/may_df_uid')
df_ven_guid_5 = load_variable('/home/jovyan/dataset/may_df_ven_guid')
df_uid51 = df_uid_5[df_uid_5.month==51]
df_ven_guid51 = df_ven_guid_5[df_ven_guid_5.month==51]

In [30]:
df_temp = pd.concat([df_uid51[['month','gid_encode']],df_ven_guid51[['month','gid_encode']]],ignore_index=True)
under = 0
freq_list = []
freq = df_temp.gid_encode.value_counts()
for i in freq.values:
    under += np.log10(i+1)
for i in freq.values:
    freq_list.append(np.log10(i+1)/under)

In [31]:
import random
def create_set_byuid(df):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby('uid'):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set

In [32]:
def create_set_byvenguid(df):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby('ven_guid'):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set

In [33]:
df_train_uid = create_set_byuid(df_uid51)
df_train_venguid = create_set_byvenguid(df_ven_guid51)
train_set501 = df_train_venguid + df_train_uid
np.random.shuffle(train_set501)
len(df_train_venguid),len(df_train_uid)

44061
45136
50699
62218
85917
93517
93554
103320
127064
394959
563315
574791


(200966, 128230)

In [34]:
def create_array(data):
    g_id = []
    hist_g_id = []
    click = []
    seq_length = []

    for index, value in enumerate(data):
        hist_g_id.append(value[1])
        seq_length.append(len(value[1]))
        g_id.append(value[2])
        click.append(value[3])  
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) != 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]

    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    click = np.array(click)
    seq_length = np.array(seq_length)
    return g_id, hist_g_id, click, seq_length
g_id_train501, hist_g_id_train501, click_train501, seq_length_train501 = create_array(train_set501)

In [39]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_le
x = {'g_id': g_id_train501, 
    'hist_g_id': hist_g_id_train501, 
    'seq_length': seq_length_train501}
y = click_train501

In [22]:
import keras
model = keras.models.load_model("/home/jovyan/dataset/rakuten_model_401_430")
checkpoint_path = "/home/jovyan/dataset/checkpoint_folder/model_401_501/rakuten_0401_0501.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history401_501 = model.fit(x, y, batch_size=128, verbose=1, epochs=10, callbacks=[cp_callback])

Epoch 1/10
2572/2572 [==============================] - 681s 255ms/step - loss: 0.5680 - accuracy: 0.7013 - auc: 0.7745

Epoch 00001: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 2/10
2572/2572 [==============================] - 657s 255ms/step - loss: 0.4785 - accuracy: 0.7719 - auc: 0.8502

Epoch 00002: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 3/10
2572/2572 [==============================] - 662s 257ms/step - loss: 0.3968 - accuracy: 0.8250 - auc: 0.9005

Epoch 00003: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 4/10
2572/2572 [==============================] - 663s 258ms/step - loss: 0.3282 - accuracy: 0.8638 - auc: 0.9332

Epoch 00004: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 5/10
2572/2572 [==============================] - 665s 259ms/step - loss: 0.2742 - accuracy: 0.8921 - auc: 0.9538

Epoch 00005: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 6/10
2572/2572 [=============

### 15個epoch

In [44]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]
x = {'g_id': g_id_train501, 
    'hist_g_id': hist_g_id_train501, 
    'seq_length': seq_length_train501}
y = click_train501

In [45]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_501")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/15epoch/401_501.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory5epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
2572/2572 [==============================] - 654s 253ms/step - loss: 0.3577 - accuracy: 0.8547 - auc: 0.9198

Epoch 00001: saving model to /home/jovyan/dataset/15epoch/401_501.ckpt
Epoch 2/5
2572/2572 [==============================] - 647s 252ms/step - loss: 0.2588 - accuracy: 0.9029 - auc: 0.9582

Epoch 00002: saving model to /home/jovyan/dataset/15epoch/401_501.ckpt
Epoch 3/5
2572/2572 [==============================] - 667s 259ms/step - loss: 0.2057 - accuracy: 0.9274 - auc: 0.9733

Epoch 00003: saving model to /home/jovyan/dataset/15epoch/401_501.ckpt
Epoch 4/5
2572/2572 [==============================] - 676s 263ms/step - loss: 0.1727 - accuracy: 0.9416 - auc: 0.9809

Epoch 00004: saving model to /home/jovyan/dataset/15epoch/401_501.ckpt
Epoch 5/5
2572/2572 [==============================] - 689s 268ms/step - loss: 0.1464 - accuracy: 0.9527 - auc: 0.9859

Epoch 00005: saving model to /home/jovyan/dataset/15epoch/401_501.ckpt


### 20 epoch

In [48]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_501")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/20epoch/model401_501/401_501.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory5epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
2572/2572 [==============================] - 702s 272ms/step - loss: 0.1257 - accuracy: 0.9609 - auc: 0.9892

Epoch 00001: saving model to /home/jovyan/dataset/20epoch/model401_501/401_501.ckpt
Epoch 2/5
2572/2572 [==============================] - 707s 275ms/step - loss: 0.1144 - accuracy: 0.9655 - auc: 0.9907

Epoch 00002: saving model to /home/jovyan/dataset/20epoch/model401_501/401_501.ckpt
Epoch 3/5
2572/2572 [==============================] - 710s 276ms/step - loss: 0.1043 - accuracy: 0.9695 - auc: 0.9920

Epoch 00003: saving model to /home/jovyan/dataset/20epoch/model401_501/401_501.ckpt
Epoch 4/5
2572/2572 [==============================] - 709s 275ms/step - loss: 0.0974 - accuracy: 0.9723 - auc: 0.9927

Epoch 00004: saving model to /home/jovyan/dataset/20epoch/model401_501/401_501.ckpt
Epoch 5/5
2572/2572 [==============================] - 709s 276ms/step - loss: 0.0871 - accuracy: 0.9758 - auc: 0.9940

Epoch 00005: saving model to /home/jovyan/dataset/20epoch/model4